In [1]:
import psycopg2
import pandas as pd
import httplib2
import apiclient.discovery
import datetime
from oauth2client.service_account import ServiceAccountCredentials

class Sqlquerry():
    '''Class to send SQL-Querry to backend DB and put result to the spreadsheet'''
    
    def __init__(self, db_querry, spreadsheetId):
        self.db_querry = db_querry
        self.spreadsheetId = spreadsheetId

        CREDENTIALS_FILE = 'testapi-263014-a34e6c59997b.json'
        self.credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_FILE,
                                                                       ['https://www.googleapis.com/auth/spreadsheets',
                                                                        'https://www.googleapis.com/auth/drive'])
        self.httpAuth = self.credentials.authorize(httplib2.Http())
        self.service = apiclient.discovery.build('sheets', 'v4', http = self.httpAuth)
        

        
    def dbquerry(self):
        '''Sending SQL-Querry to DB and returns Pandas DataFrame'''
        
        conn = psycopg2.connect("dbname='anyclass_prod' user='analytic' host='10.10.3.87' password='I59vPF59sbc6o9Lly'")
        data = pd.read_sql(self.db_querry, conn)
        self.data = data
        
        
    
    def date_transform(self):
        '''Returns reporting date, querry results and defines range of sheet to fill in'''
        
        date_reported, created_qty, created_sum, paid_qty, paid_sum = self.data.values.tolist()[0]
        
        date_reported = datetime.datetime.strptime(str(date_reported),
                                                   '%Y-%m-%d %H:%M:%S').strftime('%d.%m.%Y')                                
        
        
        date_check_result = self.service.spreadsheets().values().get(
            spreadsheetId=self.spreadsheetId, range='dataDB!A2:A').execute()
        date_check_rows = date_check_result.get('values', [])
        
        
        cell_no = 0
        
        for element in date_check_rows:
            if element[0] != date_reported:
                cell_no += 1
            else:
                cell_no += 2
                break
                
        fill_range = f'dataDB!B{cell_no}'
        
        created_qty = created_qty if created_qty else 0
        created_sum = created_sum if created_sum else 0
        paid_qty = paid_qty if paid_qty else 0
        paid_sum = paid_sum if paid_sum else 0
        
        self.date_reported, self.created_qty, self.created_sum, self.paid_qty, \
        self.paid_sum, self.fill_range = date_reported, created_qty, created_sum, \
        paid_qty, paid_sum, fill_range
        

    
    def fill_in(self):
        '''Executes values placing into sheet range defined'''
        
        current_ts = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        results = self.service.spreadsheets().values().batchUpdate(
            spreadsheetId = self.spreadsheetId, body = {
                "valueInputOption": "USER_ENTERED",
                "data": [
                    {"range": self.fill_range,
                     "majorDimension": "ROWS",
                     "values": [[current_ts, self.created_sum, self.created_qty, self.paid_sum, self.paid_qty]]}
                ]
            }).execute()
        
        
    def class_execute(self):
        '''Executes class methods'''
        self.dbquerry()
        self.date_transform()
        self.fill_in()

/opt/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


ModuleNotFoundError: No module named 'httplib2'